In [8]:
import xarray as xr
import pathlib
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
import os
from multiprocessing import Pool

In [11]:
from platform import python_version

print(python_version())

3.9.2


In [9]:
xr.__version__

'0.18.0'

In [3]:
# make your root_dir the current working directory
root_dir = Path.cwd() # set the root directory as a Pathlib path
folder_raw_data = root_dir / 'data/raw/' # raw data folder that holds the .nc files



In [7]:
Path(Path('test_dir'))

PosixPath('test_dir')

In [3]:
def plot_radial_layer(data, r_index, time_step='001', save_img=False, save_folder_name='radial_layer_img',):
    
    # if save folder doesn't exist, create it
    if save_img:
        img_folder = Path(save_folder_name)

    fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(14,8), dpi=40)
    
    color_scheme='inferno'

    for variable, ax in zip(list(data.keys()), axes.flat):
        ax.pcolormesh(data[variable].values[:,r_index,:], cmap=color_scheme)
        ax.set_title(f"{variable}")
        min_val = np.min(data[variable].values[:,r_index,:])
        max_val = np.max(data[variable].values[:,r_index,:])

        print_text = f"timestep = {time_step}\nr_index = {r_index}\nmin = {min_val:.2E}\nmax = {max_val:.2E}"

        x_min, x_max = ax.get_xlim()
        y_min, y_max = ax.get_ylim()

        ax.text(
            (x_max - x_min) * 0.03 + x_min,
            y_max - (y_max - y_min) * 0.05,
            print_text,
            fontsize=18,
            fontweight="normal",
            verticalalignment="top",
            color='white',
            horizontalalignment="left",
            bbox={"facecolor": "gray", "alpha": 0.0, "pad": 6},
        )
    
    fig.tight_layout(w_pad=2.0, h_pad=2.0)

    if save_img:
        save_name = img_folder / f'{time_step}_{r_index}.jpg'
        plt.savefig(save_name,)
        plt.close('all')
    else:
        plt.show()

In [4]:
# make your root_dir the current working directory
root_dir = Path.cwd() # set the root directory as a Pathlib path
folder_raw_data = root_dir / 'data/raw/' # raw data folder that holds the .nc files


def save_all_radial_img(file_path):
    # get time step
    time_step = re.findall('[0-9]+', str(file_path))[-1]
    
    # load data file
    data = xr.open_dataset(file_path)
    
    # get no. radial layers
    for r_index in range(data.r.values.shape[0]):
        plot_radial_layer(data, r_index, time_step, save_img=True, save_folder_name='radial_layer_img',)


def main(folder_raw_data):
    
    Path('radial_layer_img').mkdir(parents=True, exist_ok=True)
    
    # get a list of file names
    file_list = list(folder_raw_data.rglob('*.nc'))

    # set up your pool
    with Pool(processes=32) as pool: # or whatever your hardware can support

        # have your pool map the file to the radial image creator
        pool.map(save_all_radial_img, file_list)

        

if __name__ == '__main__':
    main(folder_raw_data)

Exception ignored in: Exception ignored in: <_io.BufferedReader name='/home/tvhahn/earth1/lib/python3.8/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf'><_io.BufferedReader name='/home/tvhahn/earth1/lib/python3.8/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf'>

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/tvhahn/earth1/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py", line 238, in get_text_width_height_descent
  File "/home/tvhahn/earth1/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py", line 238, in get_text_width_height_descent
        font.set_text(s, 0.0, flags=flags)font.set_text(s, 0.0, flags=flags)

OSErrorOSError: : [Errno 12] Cannot allocate memory[Errno 12] Cannot allocate memory



KeyboardInterrupt: 